In [21]:
####################################################################
# Prepared for Gabor's Data Analysis
#
# Data Analysis for Business, Economics, and Policy
# by Gabor Bekes and  Gabor Kezdi
# Cambridge University Press 2021
#
# gabors-data-analysis.com
#
# License: Free to share, modify and use for educational purposes.
# 	Not to be used for commercial purposes.
#
####################################################################

####################################################################
# hotels-vienna dataset
#
# input:
#      hotelbookingdata-vienna.csv

# output:
#      hotels-vienna.csv

# version 1.0   2021-05-23
####################################################################

In [32]:
### SETTING UP DIRECTORIES

# import packages
import pandas as pd
import os
import numpy as np

# set working directory for da_data_repo -- replace the
os.chdir("/Users/vigadam/Dropbox/work/data_book/da_data_repo")

# location folders
data_in = "./hotels-vienna/raw/"
data_out = "./hotels-vienna/clean/"

In [33]:
### IMPORT AND PREPARE DATA

# variables downoaded as string, often in form that is not helpful
# need to transform then to numbers that we can use

df = pd.read_csv(
    data_in + "hotelbookingdata-vienna.csv",
    quotechar='"',
    delimiter=",",
    encoding="utf-8",
)

In [34]:
# generate numerical variable of rating variable from string variable
#  trick: ignore charecters listed in option

#  distance to center entered as string in miles with one decimal

df["distance"] = df.center1distance.str.strip("miles \n\t").astype(float)
df["distance_alter"] = df.center2distance.str.strip("miles \n\t").astype(float)

df["accommodation_type"] = df.accommodationtype.str.split("@").str[1]
df.drop(columns=["accommodationtype"], inplace=True)

df["nnights"] = [4 if x == "price for 4 nights" else 1 for x in df["price_night"]]

# ratings
# generate numerical variable of rating variable from string variable

df["rating"] = (
    df.guestreviewsrating.str.split("/").str[0].str.strip(" \n\t").astype(float)
)

df["rating_count"] = pd.to_numeric(df["rating_reviewcount"], errors="coerce").astype(
    "Int64"
)

In [35]:
# rename variables

df.rename(
    columns={
        "rating2_ta": "ratingta",
        "rating2_ta_reviewcount": "ratingta_count",
        "addresscountryname": "country",
        "s_city": "city",
        "starrating": "stars",
    },
    inplace=True,
)

In [36]:
# ARRRANGE AND CLEAN DATASET

df.loc[df["stars"] == 0, "stars"] = np.nan

df.dropna(subset=["hotel_id"], inplace=True)

df.drop(
    columns=[
        "center2distance",
        "center1distance",
        "price_night",
        "guestreviewsrating",
        "rating_reviewcount",
    ],
    inplace=True,
)

# drop duplicates by relevant subset

df.drop_duplicates(
    subset=[
        "city",
        "hotel_id",
        "distance",
        "stars",
        "rating",
        "price",
        "year",
        "month",
        "weekend",
        "holiday",
    ],
    keep="first",
    inplace=True,
)
df = df.sort_values(
    [
        "city",
        "hotel_id",
        "distance",
        "stars",
        "rating",
        "year",
        "month",
        "weekend",
        "holiday",
    ]
)
df = df.reset_index(drop=True)

first_column = df.pop("hotel_id").astype(int)
df.insert(0, "hotel_id", first_column)

In [37]:
# save table

df.to_csv(data_out + "hotels-vienna.csv", index=False)